In [4]:
%reload_ext autoreload
%autoreload 2

import tirutils,requests,configparser,re
from IPython.display import display

config = configparser.RawConfigParser()
config.read_file(open(r'../config/stuff.py'))
gc2APIKey = config.get('apiKeys','apiKey_GC2_BCB').replace('"','')
apiBaseURL = "https://gc2.mapcentia.com/api/v1/sql/bcb?key="+gc2APIKey

wormsRecordsByNameBaseURL = "http://www.marinespecies.org/rest/AphiaRecordsByName/"
wormsRecordByAphiaIDBaseURL = "http://www.marinespecies.org/rest/AphiaRecordByAphiaID/"

In [5]:
def wormsPairEverything(wormsMatchMethod,wormsData):
    import re
    wormsPairingRules = {"{": "", "}": "", "'": '"', ": ": "=>", '=>None,': '=>"None",'}
    wormsPairs = wormsData
    for i, j in wormsPairingRules.items():
        wormsPairs = wormsPairs.replace(i, j)
    wormsPairs = re.sub(r'=>([0-9]+),', r'=>"\1",', wormsPairs)
    wormsPairs = wormsPairs+","+wormsMatchMethod
    return wormsPairs

In [6]:
targetDataSQL = "SELECT gid, \
    registrationinfo -> 'SGCN_ScientificName_Submitted' AS scientificname \
    FROM tir.tir \
    WHERE itis IS NULL \
    AND worms IS NULL \
    ORDER BY gid LIMIT 100"

targetData = requests.get(apiBaseURL+"&q="+targetDataSQL).json()

In [12]:
commitData = True

for feature in targetData['features']:
    gid = feature['properties']['gid']
    scientificname = feature['properties']['scientificname']
    numFoundExact = 0
    numFoundFuzzy = 0

    if scientificname.find("pop."):
        nameParts = scientificname.split()
        regex = re.compile(r'pop\.[0-9]')
        scientificname = ' '.join([i for i in nameParts if not regex.search(i)])
    
    if scientificname.find("spp."):
        nameParts = scientificname.split()
        regex = re.compile(r'spp\.')
        scientificname = ' '.join([i for i in nameParts if not regex.search(i)])
    
    if scientificname.find("(") and scientificname.find(")"):
        nameParts = scientificname.split()
        regex = re.compile(r'\(')
        scientificname = ' '.join([i for i in nameParts if not regex.search(i)])
    
    if any(x.isupper() for x in scientificname[-(len(scientificname)-1):]):
        scientificname = scientificname.lower().capitalize()
    
    wormsFuzzyMatchURL = wormsRecordsByNameBaseURL+scientificname+"?like=true&marine_only=false"
    try:
        wormsFuzzyMatchR = requests.get(wormsFuzzyMatchURL).json()
    except:
        continue
    if len(wormsFuzzyMatchR) == 1:
        if wormsFuzzyMatchR[0]['AphiaID'] == wormsFuzzyMatchR[0]['valid_AphiaID']:
            wormsMatchMethod = '"wormsMatchMethod"=>"found match on scientific name"'
            worms = wormsPairEverything(wormsMatchMethod,str(wormsFuzzyMatchR[0]))
        else:
            aphiaIDURL = wormsRecordByAphiaIDBaseURL+str(wormsFuzzyMatchR[0]['valid_AphiaID'])
            try:
                aphiaR = requests.get(aphiaIDURL).json()
            except:
                continue
            wormsMatchMethod = '"wormsMatchMethod"=>"found match by following valid aphiaid"'
            worms = wormsPairEverything(wormsMatchMethod,str(aphiaR))
    
    print (gid, worms)

282 "url"=>"http://www.marinespecies.org/aphia.php?p=taxdetails&id=944219", "phylum"=>"Arthropoda", "isBrackish"=>"0", "kingdom"=>"Animalia", "scientificname"=>"Abacion wilhelminae", "family"=>"Abacionidae", "match_type"=>"like", "citation"=>"Sierwald, P. (2017). Abacion wilhelminae Shelley, McAllister & Hollis, 2003. In=>Sierwald, P. MilliBase. Accessed through=> World Register of Marine Species at http://www.marinespecies.org/aphia.php?p=taxdetails&id=944219 on 2017-05-02", "modified"=>"2017-01-20T10:14:02Z", "isMarine"=>"0", "rank"=>"Species", "unacceptreason"=>None, "class"=>"Diplopoda", "lsid"=>"urn:lsid:marinespecies.org:taxname:944219", "isExtinct"=>None, "authority"=>"Shelley, McAllister & Hollis, 2003", "valid_name"=>"Abacion wilhelminae", "valid_authority"=>"Shelley, McAllister & Hollis, 2003", "isFreshwater"=>"0", "AphiaID"=>"944219", "isTerrestrial"=>"1", "order"=>"Callipodida", "genus"=>"Abacion", "status"=>"accepted", "valid_AphiaID"=>944219,"wormsMatchMethod"=>"found mat